In [1]:
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
import torch

/Users/mab/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/mab/Library/Python/3.9/lib/python/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
# tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-2.8b-hf")
# model = MambaForCausalLM.from_pretrained("state-spaces/mamba-2.8b-hf")
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-370m-hf")
model = MambaForCausalLM.from_pretrained("state-spaces/mamba-370m-hf")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


In [3]:
input_ids = tokenizer("What is mamba?", return_tensors="pt")["input_ids"]

out = model.generate(input_ids, max_new_tokens=100)
print(tokenizer.batch_decode(out))

['What is mamba?\n\nA:\n\nMamba is a genus of the family Colubridae.\n\nA:\n\nMamba is a genus of the family Colubridae.\n\nA:\n\nMamba is a genus of the family Colubridae.\n\n<|endoftext|>']


In [4]:
dataset = load_dataset("Abirate/english_quotes", split="train")
dataset

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Dataset({
    features: ['quote', 'author', 'tags'],
    num_rows: 2508
})

In [5]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=2e-3
)
lora_config =  LoraConfig(
        r=8,
        target_modules=["x_proj", "embeddings", "in_proj", "out_proj"],
        task_type="CAUSAL_LM",
        bias="none"
)

In [6]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    peft_config=lora_config,
    train_dataset=dataset,
    dataset_text_field="quote",
)

/Users/mab/Library/Python/3.9/lib/python/site-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [7]:
trainer.train()

  1%|          | 10/1881 [02:25<7:02:34, 13.55s/it]

{'loss': 14.9046, 'grad_norm': nan, 'learning_rate': 0.0019893673577884106, 'epoch': 0.02}


  1%|          | 20/1881 [05:44<7:27:15, 14.42s/it] 

{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.001978734715576821, 'epoch': 0.03}


  2%|▏         | 30/1881 [07:42<7:44:29, 15.06s/it]

...

  7%|▋         | 140/1881 [53:12<15:21:05, 31.74s/it]

{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.001851143009037746, 'epoch': 0.22}


  8%|▊         | 150/1881 [1:03:10<10:52:04, 22.60s/it] 

{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 0.0018405103668261563, 'epoch': 0.24}


  8%|▊         | 154/1881 [1:03:28<4:03:40,  8.47s/it] 